# *Pipeline de limpeza dos dados*

In [21]:
import pandas as pd
import numpy as np
import math
import re

In [22]:
#Resolvendo números em faixas --> acessar valor e substituir pela média da faixa
def calcular_media_faixa(valor):
    if isinstance(valor, str) and '-' in valor:
        try:
            parte1, parte2 = valor.split('-')
            media = (float(parte1) + float(parte2)) / 2
            return media
        except ValueError:
            return np.nan
    return valor

In [23]:
base_original = pd.read_csv("base_bruta_imoveis_zap_belo_horizonte.csv", sep = ";")
base_original.head()

,Preço,Tamanho,Quartos,Banheiros,Vagas,Bairro,Endereço
0,R$ 1.450.000,Tamanho do imóvel135 m²,Quantidade de quartos4,Quantidade de banheiros2,Quantidade de vagas de garagem2,Gutierrez,Rua Cônego Rocha Franco
1,Valor sob consulta,Tamanho do imóvel62-67 m²,Quantidade de quartos2,Quantidade de banheiros1,N/D,Funcionários,Rua Gonçalves Dias
2,R$ 1.070.000,Tamanho do imóvel115 m²,Quantidade de quartos2,Quantidade de banheiros2,Quantidade de vagas de garagem2,Santo Antônio,Rua Antônio Dias
3,R$ 280.000,Tamanho do imóvel60 m²,Quantidade de quartos3,Quantidade de banheiros1,Quantidade de vagas de garagem1,Estoril,Rua Vereador Nelson Cunha
4,A partir deR$ 1.335.000,Tamanho do imóvel78 m²,Quantidade de quartos3,Quantidade de banheiros2,Quantidade de vagas de garagem2,São Pedro,Rua Viçosa


In [24]:
base_tratada = base_original.copy()

In [25]:
base_tratada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9990 entries, 0 to 9989
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Preço      9990 non-null   object
 1   Tamanho    9990 non-null   object
 2   Quartos    9990 non-null   object
 3   Banheiros  9990 non-null   object
 4   Vagas      9990 non-null   object
 5   Bairro     9990 non-null   object
 6   Endereço   9095 non-null   object
dtypes: object(7)
memory usage: 546.5+ KB


In [26]:
round(base_tratada.describe(), 2)

,Preço,Tamanho,Quartos,Banheiros,Vagas,Bairro,Endereço
count,9990,9990,9990,9990,9990,9990,9095
unique,2099,711,15,29,26,192,1614
top,N/D,Tamanho do imóvel90 m²,Quantidade de quartos3,Quantidade de banheiros2,Quantidade de vagas de garagem2,Buritis,Rua dos Aimorés
freq,1208,264,4548,4128,4864,622,111


# *Coluna Preço*

In [27]:
base_tratada["Preço"] = base_tratada["Preço"].str.replace(r"[^\d.]", "", regex = True)
base_tratada["Preço"] = base_tratada["Preço"].str.replace(".","")
base_tratada["Preço"] = pd.to_numeric(base_tratada["Preço"], errors = "coerce")

# *Coluna Tamanho*

In [28]:
base_tratada["Tamanho"] = base_tratada["Tamanho"].str.replace("Tamanho do imóvel", "").str.replace("m²", "")
base_tratada["Tamanho"] = pd.to_numeric(base_tratada["Tamanho"], errors = "coerce")

# *Coluna Quartos*

In [29]:
base_tratada["Quartos"] = base_tratada["Quartos"].str.replace("Quantidade de quartos","")
base_tratada["Quartos"] = base_tratada["Quartos"].apply(calcular_media_faixa)
base_tratada["Quartos"] = pd.to_numeric(base_tratada["Quartos"], errors = "coerce")
base_tratada["Quartos"] = base_tratada["Quartos"].apply(lambda x: math.ceil(x) if pd.notna(x) else x)
base_tratada.loc[base_tratada["Quartos"] > 7, "Quartos"] = 8

# *Coluna Banheiros*

In [30]:
base_tratada["Banheiros"] = base_tratada["Banheiros"].str.replace("Quantidade de banheiros","")
base_tratada["Banheiros"] = base_tratada["Banheiros"].apply(calcular_media_faixa)
base_tratada["Banheiros"] = pd.to_numeric(base_tratada["Banheiros"], errors = "coerce")
base_tratada["Banheiros"] = base_tratada["Banheiros"].apply(lambda x: math.ceil(x) if pd.notna(x) else x)
base_tratada.loc[base_tratada["Banheiros"] > 6, "Banheiros"] = 7

# *Coluna Vagas*

In [31]:
base_tratada["Vagas"] = base_tratada["Vagas"].str.replace("Quantidade de vagas de garagem","")
base_tratada["Vagas"] = base_tratada["Vagas"].replace("N/D", "0")
base_tratada["Vagas"] = base_tratada["Vagas"].apply(calcular_media_faixa)
base_tratada["Vagas"] = pd.to_numeric(base_tratada["Vagas"], errors="coerce")
base_tratada["Vagas"] = base_tratada["Vagas"].apply(lambda x: math.ceil(x) if pd.notna(x) else x)
base_tratada.loc[base_tratada["Vagas"] > 5, "Vagas"] = 6

# *Coluna Bairro*

In [32]:
base_tratada["Bairro"] = (
    base_tratada["Bairro"]
    .str.replace("Apartamento para comprar em", "", regex=False)
    .str.replace("Cobertura para comprar em", "", regex=False)
    .str.replace(", Belo Horizonte", "", regex=False)
    .str.replace("Imóvel para comprar em", "", regex=False)
    .str.replace("Duplex para comprar em", "", regex=False)
    .str.replace("Flat para comprar em", "", regex=False)
    .str.replace("Casa para comprar em", "", regex=False)
    .str.replace(r"\s*\(.*?\)", "", regex=True)
    .str.strip())

# *Coluna Endereço*

In [33]:
base_tratada["Endereço"] = base_tratada["Endereço"].replace(r'^\s*$', np.nan, regex=True)

# *Removendo NANs e Duplicatas*

In [34]:
base_tratada = base_tratada.dropna()
base_tratada = base_tratada.drop_duplicates()

# *Renomeando Colunas*

In [35]:
base_tratada.rename(columns={"Tamanho": "tamanho (m²)",
                             "Quartos": "qtd_quartos",
                             "Banheiros": "qtd_banheiros",
                             "Vagas": "qtd_vagas_garagem",
                             "Preço": "preco",
                             "Bairro": "bairro",
                             "Endereço": "endereco"},
                               inplace = True)

# *Removendo e salvando outliers*


In [36]:
#Limite máximo de preço
limite_maximo_preco = 5_000_000

#Limite máximo de tamanho
limite_maximo_tamanho = 600

#Identificar outliers (apenas pelo limite superior) ---
outliers = (
    (base_tratada["preco"] > limite_maximo_preco) |
    (base_tratada["tamanho (m²)"] > limite_maximo_tamanho)
)

outliers_df = base_tratada[outliers]
outliers_df.to_csv("outliers_removidos.csv", index=False, encoding="utf-8-sig")

base_tratada = base_tratada[~outliers]

print(f"Arquivo 'outliers_removidos.csv' salvo com {len(outliers_df)} linhas.")

Arquivo 'outliers_removidos.csv' salvo com 64 linhas.


# *Informações da base tratada*

In [37]:
base_tratada.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6885 entries, 0 to 9978
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   preco              6885 non-null   float64
 1   tamanho (m²)       6885 non-null   float64
 2   qtd_quartos        6885 non-null   int64  
 3   qtd_banheiros      6885 non-null   float64
 4   qtd_vagas_garagem  6885 non-null   int64  
 5   bairro             6885 non-null   object 
 6   endereco           6885 non-null   object 
dtypes: float64(3), int64(2), object(2)
memory usage: 430.3+ KB


In [38]:
round(base_tratada.describe(), 2)

,preco,tamanho (m²),qtd_quartos,qtd_banheiros,qtd_vagas_garagem
count,6885.00,6885.00,6885.00,6885.00,6885.00
mean,1107011.39,112.59,2.93,2.39,2.00
std,759315.94,60.28,0.83,1.03,0.95
min,140000.00,10.00,1.00,1.00,0.00
25%,600000.00,70.00,2.00,2.00,1.00
50%,870000.00,97.00,3.00,2.00,2.00
75%,1350300.00,140.00,4.00,3.00,2.00
max,5000000.00,600.00,7.00,7.00,6.00


# *Salvando base de dados tratada*

In [39]:
base_tratada.to_csv("base_tratada_imoveis_zap_belo_horizonte.csv", sep = ";", encoding = "utf-8-sig", index = False)

In [40]:
base_tratada

,preco,tamanho (m²),qtd_quartos,qtd_banheiros,qtd_vagas_garagem,bairro,endereco
0,1450000.0,135.0,4,2.0,2,Gutierrez,Rua Cônego Rocha Franco
2,1070000.0,115.0,2,2.0,2,Santo Antônio,Rua Antônio Dias
3,280000.0,60.0,3,1.0,1,Estoril,Rua Vereador Nelson Cunha
4,1335000.0,78.0,3,2.0,2,São Pedro,Rua Viçosa
6,530000.0,75.0,3,2.0,2,Castelo,Avenida Miguel Perrela
...,...,...,...,...,...,...,...
9957,330000.0,54.0,2,1.0,1,Juliana,Rua Alva
9963,715000.0,71.0,3,2.0,2,Padre Eustáquio,Rua Curupaiti
9971,700000.0,140.0,4,3.0,2,Alto Caiçaras,Rua da Coleirinha
9974,390000.0,49.0,2,2.0,1,Glória,Avenida Brigadeiro Eduardo Gomes
